# Predicting Particulate Matter (PM2.5) Concentrations in the Air of China

# Implementation

In [23]:
# Import libraries necessary for this project
import csv
import numpy as np
import pandas as pd
from IPython.display import display # Allows the use of display() for DataFrames
from time import time
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from math import sqrt
# Dara preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from math import sqrt
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
# Import train_test_split
from sklearn.model_selection import train_test_split


In [24]:
# Load Guangzhou dataset
Guangzhou = pd.read_csv('dataset_Guangzhou_clean.csv')
Guangzhou = Guangzhou.drop('Unnamed: 0', axis = 1)

# Load Beijing dataset
Beijing = pd.read_csv('dataset_Beijing_clean.csv')
Beijing = Beijing.drop('Unnamed: 0', axis = 1)

# Load Chengdu dataset
Chengdu = pd.read_csv('dataset_Chengdu_clean.csv')
Chengdu = Chengdu.drop('Unnamed: 0', axis = 1)

# Load Shanghai dataset
Shanghai = pd.read_csv('dataset_Shanghai_clean.csv')
Shanghai = Shanghai.drop('Unnamed: 0', axis = 1)

# Load Shenyang dataset
Shenyang = pd.read_csv('dataset_Shenyang_clean.csv')
Shenyang = Shenyang.drop('Unnamed: 0', axis = 1)

# Construct array of our datasets
datasets = {"Beijing": Beijing, "Chengdu": Chengdu, "Shanghai" : Shanghai, "Shenyang": Shenyang, "Guangzhou" : Guangzhou}
#print(datasets)

# Success - Display the first 3 records
display(Guangzhou.head(n=3))
display(Beijing.head(n=3))
display(Chengdu.head(n=3))
display(Shanghai.head(n=3))
display(Shenyang.head(n=3))


# Total number of records
n_records = len(Guangzhou["PM_US Post"])
print("Number of records for Guangzhou: ", n_records)
print("*********************")
display(Guangzhou.info())

# Total number of records
n_records = len(Beijing["PM_US Post"])
print("Number of records for Beijing: ", n_records)
print("*********************")
display(Beijing.info())

# Total number of records
n_records = len(Chengdu["PM_US Post"])
print("Number of records for Chengdu: ", n_records)
print("*********************")
display(Chengdu.info())

# Total number of records
n_records = len(Shanghai["PM_US Post"])
print("Number of records for Shanghai: ", n_records)
print("*********************")
display(Shanghai.info())

# Total number of records
n_records = len(Shenyang["PM_US Post"])
print("Number of records for Shenyang: ", n_records)
print("*********************")
display(Shenyang.info())



,PM_US Post,DEWP,HUMI,PRES,TEMP,precipitation,new_wind,year,hour_sin,hour_cos,...,season_2,season_3,season_4,cbwd_NE,cbwd_NW,cbwd_SE,cbwd_SW,cbwd_cv,Weekdays,Weekends
0,83.0,3.7,91.0,1014.2,5.0,0.0,1.9,2013,0.000000,1.000000,...,0,0,1,0,1,0,0,0,1,0
1,95.0,4.2,88.0,1014.2,6.0,0.0,1.5,2013,0.258819,0.965926,...,0,0,1,0,1,0,0,0,1,0
2,94.0,4.3,84.0,1014.0,6.8,0.0,1.6,2013,0.500000,0.866025,...,0,0,1,0,1,0,0,0,1,0


,PM_US Post,DEWP,HUMI,PRES,TEMP,precipitation,new_wind,year,hour_sin,hour_cos,...,season_2,season_3,season_4,cbwd_NE,cbwd_NW,cbwd_SE,cbwd_cv,Weekdays,Weekends,cbwd_SW
0,31.0,-10.0,67.0,1018.0,-5.0,0.0,4.02,2013,0.000000,1.000000,...,0,0,1,0,1,0,0,1,0,0
1,32.0,-11.0,73.0,1017.0,-7.0,0.0,4.02,2013,0.258819,0.965926,...,0,0,1,0,1,0,0,1,0,0
2,21.0,-11.0,73.0,1017.0,-7.0,0.0,1.79,2013,0.500000,0.866025,...,0,0,1,0,1,0,0,1,0,0


,PM_US Post,DEWP,HUMI,PRES,TEMP,precipitation,new_wind,year,hour_sin,hour_cos,...,season_2,season_3,season_4,cbwd_NE,cbwd_NW,cbwd_SE,cbwd_SW,cbwd_cv,Weekdays,Weekends
0,129.0,-4.0,64.42,1022.0,2.0,0.0,1.0,2013,0.000000e+00,1.000000,...,0,0,1,0,0,0,0,1,1,0
1,135.0,-3.0,80.19,1022.0,0.0,0.0,0.0,2013,2.588190e-01,0.965926,...,0,0,1,0,0,0,0,1,1,0
2,189.0,-5.0,42.10,1020.0,7.0,0.0,2.0,2013,1.224647e-16,-1.000000,...,0,0,1,1,0,0,0,0,1,0


,PM_US Post,DEWP,HUMI,PRES,TEMP,precipitation,new_wind,year,hour_sin,hour_cos,...,season_2,season_3,season_4,cbwd_NE,cbwd_NW,cbwd_SE,cbwd_SW,cbwd_cv,Weekdays,Weekends
0,70.0,-5.0,69.0,1023.0,0.0,0.0,3.0,2013,0.000000,1.000000,...,0,0,1,0,0,0,1,0,1,0
1,76.0,-5.0,69.0,1023.0,0.0,0.0,2.0,2013,0.258819,0.965926,...,0,0,1,0,0,0,1,0,1,0
2,72.0,-5.0,69.0,1023.0,0.0,0.0,1.0,2013,0.500000,0.866025,...,0,0,1,0,0,0,1,0,1,0


,PM_US Post,DEWP,HUMI,PRES,TEMP,precipitation,new_wind,year,hour_sin,hour_cos,...,season_2,season_3,season_4,cbwd_NE,cbwd_NW,cbwd_SE,cbwd_SW,cbwd_cv,Weekdays,Weekends
0,113.0,-1.0,25.89,1013.0,19.0,0.0,10.0,2013,-0.258819,-0.965926,...,0,0,0,0,0,0,1,0,1,0
1,74.0,-6.0,26.11,1012.0,13.0,0.0,7.0,2013,0.707107,-0.707107,...,0,0,0,0,0,0,1,0,1,0
2,59.0,-5.0,26.40,1012.0,14.0,0.0,5.0,2013,0.500000,-0.866025,...,0,0,0,0,0,0,1,0,1,0


...

None

Number of records for Shanghai:  23534
*********************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23534 entries, 0 to 23533
Data columns (total 25 columns):
PM_US Post       23534 non-null float64
DEWP             23534 non-null float64
HUMI             23534 non-null float64
PRES             23534 non-null float64
TEMP             23534 non-null float64
precipitation    23534 non-null float64
new_wind         23534 non-null float64
year             23534 non-null int64
hour_sin         23534 non-null float64
hour_cos         23534 non-null float64
month_sin        23534 non-null float64
month_cos        23534 non-null float64
day_sin          23534 non-null float64
day_cos          23534 non-null float64
season_1         23534 non-null int64
season_2         23534 non-null int64
season_3         23534 non-null int64
season_4         23534 non-null int64
cbwd_NE          23534 non-null int64
cbwd_NW          23534 non-null int64
cbwd_SE          23534 non-null int64
cbwd_S

None

Number of records for Shenyang:  20452
*********************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20452 entries, 0 to 20451
Data columns (total 25 columns):
PM_US Post       20452 non-null float64
DEWP             20452 non-null float64
HUMI             20452 non-null float64
PRES             20452 non-null float64
TEMP             20452 non-null float64
precipitation    20452 non-null float64
new_wind         20452 non-null float64
year             20452 non-null int64
hour_sin         20452 non-null float64
hour_cos         20452 non-null float64
month_sin        20452 non-null float64
month_cos        20452 non-null float64
day_sin          20452 non-null float64
day_cos          20452 non-null float64
season_1         20452 non-null int64
season_2         20452 non-null int64
season_3         20452 non-null int64
season_4         20452 non-null int64
cbwd_NE          20452 non-null int64
cbwd_NW          20452 non-null int64
cbwd_SE          20452 non-null int64
cbwd_S

None

In [25]:
# Machine learning algorithms decleration
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsRegressor


LR = LinearRegression()
RF = RandomForestRegressor(n_estimators = 96)
ANN = MLPRegressor(hidden_layer_sizes= (128, 256))
SVR = SVR(kernel='rbf')
GNB = GaussianNB()
KNN = KNeighborsRegressor(n_neighbors=5)
#SVR_tuned = SVR(kernel='rbf', C = 707, epsilon = 4)

MLs = {'LR' : LR, 'RF': RF, 'ANN' : ANN, 'SVR' : SVR}
#MLs = {'LR' : LR, 'GNB' : GNB, 'KNN': KNN ,'RF': RF, 'ANN' : ANN, 'SVR' : SVR}
#MLs = {'RF' : RF}


In [26]:


def applyTTS(cityName,dataset,MLname,estimator, f_out, Un_needed_columns):
    # Construct the pipeline with a standard scaler and a small neural network
    estimators = []
    estimators.append(('standardize', StandardScaler()))
    estimators.append((MLname, estimator))
    model = Pipeline(estimators)

    # Split the data into features and target label
    #Un_needed_columns = ['PM_US Post', 'precipitation','day_cos', 'day_sin']
    
    # Split the data into features and target label
    Features = dataset.drop(Un_needed_columns, axis = 1)
    Target = dataset['PM_US Post']
    
    features_list = list(Features.columns)

    # Train Test Split 
    X_train, X_test, y_train, y_test = train_test_split(Features, Target, test_size=0.20, random_state=0)
    
    # Feature Scaling
    #Sc_X = StandardScaler()
    #X_train = Sc_X.fit_transform(X_train)
    #X_test = Sc_X.transform(X_test)
    

    print(estimators)

    print("**Train Split **")
    model.fit(X_train, y_train)
    predict = model.predict(X_test)
    R2 = r2_score(y_test, predict)
    MSE =  mean_squared_error(y_test,predict)
    MAE =  mean_absolute_error(y_test,predict)
    RMSE = sqrt(MSE)


    print("MSE : ", MSE)
    print("MAE : ", MAE)
    print("R2 : ", R2)
    print("RMSE : ", RMSE)

    print("-----------------------------------------------")

    
    f_out.write(str(cityName) + ",")
    f_out.write(str(cityName) + ",")
    f_out.write(str(MLname) + ",")
    f_out.write('TTS' + ",")
    f_out.write(str(MAE) + ",")
    f_out.write(str(MSE) + ",")
    f_out.write(str(RMSE) + ",")
    f_out.write(str(R2) + ",")
    f_out.write(str(len(features_list)) + ",")
    for feature in features_list:
        f_out.write(feature + " | ")
    f_out.write("\n")



In [27]:


def applyCV(cityName,dataset,MLname,estimator, f_out, Un_needed_columns):
    # Construct the pipeline with a standard scaler and a small neural network
    estimators = []
    estimators.append(('standardize', StandardScaler()))
    estimators.append((MLname, estimator))
    model = Pipeline(estimators)

    # Split the data into features and target label
    #Un_needed_columns = ['PM_US Post']
    
    # Split the data into features and target label
    X = dataset.drop(Un_needed_columns, axis = 1)
    y = dataset['PM_US Post']

    # Saving feature names for later use
    features_list = list(X.columns)

    # We'll use 5-fold cross validation. That is, a random 80% of the data will be used
    # to train the model, and the prediction score will be computed on the remaining 20%.
    # This process is repeated five times such that the training sets in each "fold"
    # are mutually orthogonal.
    
    K = 3
    kfold = KFold(n_splits=K,  shuffle=True)

    print(estimators)

    print("**cross_val_score + KFold **")

    results_R2 = cross_val_score(model, X, y, cv=kfold, scoring='r2')
    R2 = np.mean(results_R2)
    print('CV Scoring Result: r2 : mean=',np.mean(results_R2),'std=',np.std(results_R2))
    #print(results_R2) 
    print("**************")
    
    results_MAE = cross_val_score(model, X, y, cv=kfold, scoring='neg_mean_absolute_error')
    MAE = np.mean(results_MAE)
    print('CV Scoring Result: MAE : mean=',np.mean(results_MAE),'std=',np.std(results_MAE))
    #print(results_MAE)  
    print("**************")

    results_MSE = cross_val_score(model, X, y, cv=kfold, scoring='neg_mean_squared_error')
    MSE = np.mean(results_MSE)
    print('CV Scoring Result: MSE : mean=',np.mean(results_MSE),'std=',np.std(results_MSE))
    
    
    RMSE = sqrt(abs(MSE))

    print("-----------------------------------------------")

    
    f_out.write(str(cityName) + ",")
    f_out.write(str(cityName) + ",")
    f_out.write(str(MLname) + ",")
    f_out.write('CV(' + str(K) + "),")
    f_out.write(str(abs(MAE)) + ",")
    f_out.write(str(abs(MSE)) + ",")
    f_out.write(str(RMSE) + ",")
    f_out.write(str(R2) + ",")
    f_out.write(str(len(features_list)) + ",")
    for feature in features_list:
        f_out.write(feature + "&")
    f_out.write("\n")



In [30]:
with open("../China/China_Results/Paper_Results/Level_0_Balanced_Test_21_features.csv", 'w') as f_out:
    out_colnames = ['Train Site', 'Test Site', 'Algorithm', 'CV', 'MAE', 'MSE', 'RMSE', 'R^2', 'Features_Count', 'Features']        
    writer = csv.DictWriter(f_out, fieldnames = out_colnames)
    writer.writeheader()
    
    for MLname, ML in MLs.items():
        for cityName, dataset in datasets.items(): 
            print(cityName, " : \n*******************************************")
            Un_needed_columns = ['PM_US Post','precipitation', 'Weekdays', 'Weekends']
            applyTTS(cityName,dataset,MLname,ML, f_out, Un_needed_columns)
            applyCV(cityName,dataset,MLname,ML, f_out, Un_needed_columns)
            
    
    

Beijing  : 
*******************************************
[('standardize', StandardScaler(copy=True, with_mean=True, with_std=True)), ('LR', LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False))]
**Train Split **
MSE :  4904.282409834953
MAE :  49.56083706418759
R2 :  0.4434012375610562
RMSE :  70.0305819612757
-----------------------------------------------
[('standardize', StandardScaler(copy=True, with_mean=True, with_std=True)), ('LR', LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False))]
**cross_val_score + KFold **
CV Scoring Result: r2 : mean= 0.4405373095792913 std= 0.004902849533259499
**************


/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.6/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:331: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.6/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype int64, float64 were all converted to floa

CV Scoring Result: MAE : mean= -50.030930484176785 std= 0.32892306802216825
**************
CV Scoring Result: MSE : mean= -4963.308749490324 std= 100.3409547425133
-----------------------------------------------
Chengdu  : 
*******************************************
[('standardize', StandardScaler(copy=True, with_mean=True, with_std=True)), ('LR', LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False))]
**Train Split **
MSE :  2074.9289034744743
MAE :  33.469266460066386
R2 :  0.40537002580256
RMSE :  45.5513875032855
-----------------------------------------------
[('standardize', StandardScaler(copy=True, with_mean=True, with_std=True)), ('LR', LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False))]
**cross_val_score + KFold **


/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.6/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:331: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.6/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype int64, float64 were all converted to floa

CV Scoring Result: r2 : mean= 0.39754401255580757 std= 0.0113537700573936
**************
CV Scoring Result: MAE : mean= -33.45724916458054 std= 0.2889638310626188
**************


...

CV Scoring Result: r2 : mean= 0.31202445945535606 std= 0.008413728461762736
**************


/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.6/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:331: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.6/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype int64, float64 were all converted to floa

CV Scoring Result: MAE : mean= -18.484136540742607 std= 0.06778967020449318
**************


/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.6/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:331: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.6/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype int64, float64 were all converted to floa

CV Scoring Result: MSE : mean= -783.6067011529911 std= 38.79500783658429
-----------------------------------------------
